In [6]:
import config as cfg
import requests
import time
import urllib.parse
import hashlib
import hmac
import base64
import json
import plotly.graph_objects as go


api_url = "https://api.kraken.com"

api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
}

# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

# Function to make Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

#Function to make more non-trivial Kraken API get request
def kraken_get_request(uri_path, data=None, headers=None):
    if headers is None:
        headers = {
            'Accept': 'application/json'
        }
    
    headers.update(headers)
    req = requests.get((api_url + uri_path), headers=headers, data=data)
    return req

api_key = cfg.api_key
api_sec = cfg.api_priv

# Function to generate a nonce
def generate_nonce():
    return str(int(1000 * time.time()))


# Function to get the balance of all assets in the Kraken account with the given API keys
def grab_ext_bal():
    # Construct the Kraken API request and get the External Balance Information
    resp = kraken_request(api_endpoints['ExtendedBalance'], {"nonce": generate_nonce(),}, api_key, api_sec).json()['result']
    
    balanceDict = {}
    # Extract the balance of each non zero asset
    for asset in resp:
        if float(resp[asset]['balance']) == 0:
            continue
        balance = resp[asset]['balance']
        balanceDict[asset] = balance

    return balanceDict
balance = grab_ext_bal()
print(balance)

def grab_all_assets():
    # Grab all asset pair names from the Kraken API
    # assetPairs = requests.get(api_url + '/0/public/AssetPairs').json()
    # assetPairs = assetPairs['result']
    # altNames = [details['altname'] for details in assetPairs.values()]
    # altNames = list(assetPairs.keys())
    # return altNames

    # Construct the Kraken API request and get all asset pairs from the Kraken API
    assetPairs = kraken_get_request(api_endpoints['AssetPairs']).json()['result']
    
    # Extract 'altname' for each asset pair
    altNames = [details['altname'] for details in assetPairs.values()]
    
    print("altNames: ")
    print(altNames)
    return altNames

assetPairs = grab_all_assets()

# Extract labels and values for pie chart in plotly from the balance dictionary
labels = list(balance.keys())
values = list(balance.values())

# Create a pie chart using plotly
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()


# sns.set_theme(style='darkgrid')

# # Define seaborn color palette
# sns.set_palette('light:#5A9', n_colors=len(balanceDict))



# # Extract labels and values
# labels = list(balanceDict.keys())
# values = list(balanceDict.values())

# fig = plt.figure(figsize=(16, 16), dpi=1600)

# # Initialize 9 subplots 3x3
# fig, ax = plt.subplots(3, 3 , figsize=(12, 12), dpi=1600)
# # Plot pie chart

# ax[1,1].pie(values, labels=labels, autopct='%1.1f%%')
# plt.title('Asset Distribution')
# plt.show()



{'ATOM.F': '5.20928100', 'MSOL': '0.6047700000', 'SOL.F': '1.1884947880', 'ZGBP': '3.8113'}
altNames: 
['1INCHEUR', '1INCHUSD', 'AAVEETH', 'AAVEEUR', 'AAVEGBP', 'AAVEUSD', 'AAVEXBT', 'ACAEUR', 'ACAUSD', 'ACHEUR', 'ACHUSD', 'ADAAUD', 'ADAETH', 'ADAEUR', 'ADAGBP', 'ADAUSD', 'ADAUSDT', 'ADAXBT', 'ADXEUR', 'ADXUSD', 'AEVOEUR', 'AEVOUSD', 'AGLDEUR', 'AGLDUSD', 'AIREUR', 'AIRUSD', 'AKTEUR', 'AKTUSD', 'ALCXEUR', 'ALCXUSD', 'ALGOETH', 'ALGOEUR', 'ALGOGBP', 'ALGOUSD', 'ALGOUSDT', 'ALGOXBT', 'ALICEEUR', 'ALICEUSD', 'ALPHAEUR', 'ALPHAUSD', 'ALTEUR', 'ALTUSD', 'ANKREUR', 'ANKRUSD', 'ANKRXBT', 'ANTETH', 'ANTEUR', 'ANTUSD', 'ANTXBT', 'APEEUR', 'APEUSD', 'APEUSDT', 'API3EUR', 'API3USD', 'APTEUR', 'APTUSD', 'ARBEUR', 'ARBUSD', 'ARKMEUR', 'ARKMUSD', 'ARPAEUR', 'ARPAUSD', 'ASTREUR', 'ASTRUSD', 'ATLASEUR', 'ATLASUSD', 'ATOMETH', 'ATOMEUR', 'ATOMGBP', 'ATOMUSD', 'ATOMUSDT', 'ATOMXBT', 'AUCTIONEUR', 'AUCTIONUSD', 'AUDIOEUR', 'AUDIOUSD', 'AUDJPY', 'AUDUSD', 'AVAXEUR', 'AVAXUSD', 'AVAXUSDT', 'AXSEUR', 'AXSUS

In [7]:
# Function to get the balance of all assets in the Kraken account with the given API keys
def grab_ext_bal():
    # Construct the Kraken API request and get the External Balance Information
    resp = kraken_request(api_endpoints['ExtendedBalance'], {"nonce": generate_nonce(),}, api_key, api_sec).json()
    print(resp)
    balanceDict = {}
    # Extract the balance of each asset and the asset name
    for asset, details in resp['result'].items():
        balance = details['balance']
        if float(balance) == 0:
            continue   
        balanceDict[asset] = float(balance)

    return balanceDict

assetPairs = grab_all_assets()
balanceDict = grab_ext_bal()
print(balanceDict)

altNames: 
['1INCHEUR', '1INCHUSD', 'AAVEETH', 'AAVEEUR', 'AAVEGBP', 'AAVEUSD', 'AAVEXBT', 'ACAEUR', 'ACAUSD', 'ACHEUR', 'ACHUSD', 'ADAAUD', 'ADAETH', 'ADAEUR', 'ADAGBP', 'ADAUSD', 'ADAUSDT', 'ADAXBT', 'ADXEUR', 'ADXUSD', 'AEVOEUR', 'AEVOUSD', 'AGLDEUR', 'AGLDUSD', 'AIREUR', 'AIRUSD', 'AKTEUR', 'AKTUSD', 'ALCXEUR', 'ALCXUSD', 'ALGOETH', 'ALGOEUR', 'ALGOGBP', 'ALGOUSD', 'ALGOUSDT', 'ALGOXBT', 'ALICEEUR', 'ALICEUSD', 'ALPHAEUR', 'ALPHAUSD', 'ALTEUR', 'ALTUSD', 'ANKREUR', 'ANKRUSD', 'ANKRXBT', 'ANTETH', 'ANTEUR', 'ANTUSD', 'ANTXBT', 'APEEUR', 'APEUSD', 'APEUSDT', 'API3EUR', 'API3USD', 'APTEUR', 'APTUSD', 'ARBEUR', 'ARBUSD', 'ARKMEUR', 'ARKMUSD', 'ARPAEUR', 'ARPAUSD', 'ASTREUR', 'ASTRUSD', 'ATLASEUR', 'ATLASUSD', 'ATOMETH', 'ATOMEUR', 'ATOMGBP', 'ATOMUSD', 'ATOMUSDT', 'ATOMXBT', 'AUCTIONEUR', 'AUCTIONUSD', 'AUDIOEUR', 'AUDIOUSD', 'AUDJPY', 'AUDUSD', 'AVAXEUR', 'AVAXUSD', 'AVAXUSDT', 'AXSEUR', 'AXSUSD', 'BADGEREUR', 'BADGERUSD', 'BALEUR', 'BALUSD', 'BALXBT', 'BANDEUR', 'BANDUSD', 'BATETH', 

In [ ]:
sns.set_theme(style='darkgrid')

# Define seaborn color palette
sns.set_palette('light:#5A9', n_colors=len(balanceDict))



# Extract labels and values
labels = list(balanceDict.keys())
values = list(balanceDict.values())

fig = plt.figure(figsize=(16, 16), dpi=1600)

# Initialize 9 subplots 3x3
fig, ax = plt.subplots(3, 3 , figsize=(12, 12), dpi=1600)
# Plot pie chart

ax[1,1].pie(values, labels=labels, autopct='%1.1f%%')
plt.title('Asset Distribution')
plt.show()





ledgerTypes = ['all']
print(','.join(ledgerTypes))
# Construct the Kraken API request and get ledger information
resp = kraken_request(api_endpoints['Ledgers'], {
        "nonce": generate_nonce(),
        "type": 'all',
        "start": 1695728276,
        "end": 1695828276,
        
    }, api_key, api_sec)

    # Convert the response JSON to a Python dictionary
data = resp.json()
print("Ledger Information: ", data)

'''
# Convert to spot pair names
for i in range(len(portfolioPairs)):
    portfolioPairs[i] = portfolioPairs[i] + "USD"
'''


In [ ]:
api_endpoints = {
    'Assets': '/public/Assets',
    'AssetPairs': '/public/AssetPairs',
    'Ticker': '/public/Ticker',
    'OHLC': '/public/OHLC',

    'Balance': '/private/Balance',
    'ExtendedBalance': '/private/BalanceEx',
    'Ledgers': '/private/Ledgers',
    'QueryLedgers': '/private/QueryLedgers',
    'TradeVolume': '/private/TradeVolume',
}

print("api_endpoints: ", api_url + api_endpoints['Assets'])

In [ ]:
import requests
import json

url = "https://api.kraken.com/0/private/QueryLedgers"

payload = json.dumps({
  "nonce": 1695828490,
  "id": "LBRWPL-NM5KL-YLWZNU, LBTD5Q-H23DC-YRUPVY",
  "trade": False
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'API-Key': 'jUctxyUKBrXM4MqGIbhtjLGZVG0zs3pJSmdX7IDZRF44SmUZwVGr+MY4',
  'API-Sign': 'FIZ/RPuw4ZCDujZ7m8G0Ige+WwfUyGxTSchO73goSmPtIe4ZSABakZpFQy3VTLpqKqo5aaqXyaQmi+5jymk26A=='
}

response = requests.request("POST", url, headers=headers, data=payload).json()

print(response['result'])